In [3]:
%load_ext autoreload
%autoreload

import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import ipywidgets as widgets

from IPython.display import clear_output

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from anomalyzer.config import loader
from anomalyzer.evaluate_performance import evaluate_performance
from anomalyzer.prepare_data import prepare_data

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import KFold

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope

# Which day of the week do you want to analyze?

In [4]:
selection = widgets.Dropdown(
    options=['Tuesday'],
    description='Day of Week:',
    disabled=False,
)
display(selection)

Dropdown(description='Day of Week:', options=('Tuesday',), value='Tuesday')

## What fraction of total network traffic in the selected day do you want to inspect?

Note: All network traffic might take some time to load. Suggested value is 0.1

In [5]:
sample_size =widgets.FloatSlider(
    value=0.1,
    min=0,
    max=1,
    step=0.01,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

display(sample_size)

FloatSlider(value=0.1, continuous_update=False, description='Test:', max=1.0, step=0.01)

In [8]:
go_button =widgets.Button(
    description='INSPECT',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to perform anomaly detection',
    icon='check'
)
go_output = widgets.Output()

def go_button_clicked(b):
    with go_output:
        clear_output()
        anomaly_detector(selection,sample_size)
        
go_button.on_click(go_button_clicked)

display(go_button,go_output)

Button(button_style='success', description='INSPECT', icon='check', style=ButtonStyle(), tooltip='Click to per…

Output()

In [7]:
def anomaly_detector(selection, sample_size, model='svm', new_dimension=25, predict=False):
    


    day_of_week = selection.value
    frac = sample_size.value 

    data_file= loader(day_of_week=day_of_week,data_type='processed',subtype='Normalized')
    df=pd.read_pickle(data_file)

    X,y=prepare_data(data=df,class_column='Label',classes='binary',neg_class='BENIGN')

    pca_model = PCA(n_components=new_dimension, random_state=4162)
    pca_model.fit(X)
    x_red_pca = pca_model.transform(X)

    if predict:
        filename='./models/'+str(model)+'_model_'+day_of_week+'_PCA_'+str(new_dimension)+'.pkl'

        with open(filename,'rb') as file:
            anomaly_model=pickle.load(file)
        
        anomalies = anomaly_model.predict(x_red_pca)
    else:
        filename='./predictions/'+model+'_anomalies_'+day_of_week+'_PCA_'+str(new_dimension)+'.pkl'
        with open(filename,'rb') as file:
            anomalies=pickle.load(file) 
        
    anomalies = (anomalies+1)*0.5
    anomalies = anomalies.astype(int)
        
    visuals_file='./tsne_transforms/day_of_week_perplexity_30_data_445645.pkl'

    with open(visuals_file,'rb') as file:
        x_vis=pickle.load(file)
        
    visual_sample = int(x_vis.shape[0]*frac)
    index = np.random.randint(0,x_vis.shape[0],visual_sample)
    x_red_vis = x_vis[index,:]    
        
    fig1=plt.figure(figsize=(15,10))
    sns.scatterplot(x_red_vis[:,0],x_red_vis[:,1],hue=anomalies[index],hue_order=[1,0])
    plt.title('Detected Anomalies for '+day_of_week+' Network Traffic')
    plt.legend(['Anomaly','Normal'])
    plt.show()
    
#     fig=plt.figure(num=2,figsize=(15,10))
#     sns.scatterplot(x_vis[:,0],x_vis[:,1],hue=y,hue_order=[0,1])
#     plt.title('Actual Attacks for '+day_of_week+' Network Traffic')
#     plt.legend(['Anomaly','Normal'])
#     plt.show()